# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('tic-tac-toe.csv')

# Display the first few rows to inspect
df.head()


In [ ]:
# Convert categorical columns to numeric
df = df.replace({'x': 1, 'o': 0, 'b': -1})

# Display the converted DataFrame
df.head()


In [ ]:
# Separate inputs (X values) and output (y values)
X = df.iloc[:, :-1].values  # All columns except the last
y = df.iloc[:, -1].values   # Last column is the result (Player X win/loss)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the input data
X_normalized = scaler.fit_transform(X)

# Check the normalized data
print(X_normalized[:5])  # Show the first few rows


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Check the shapes of the splits
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
pip install --upgrade tensorflow


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create the Sequential model
model = Sequential()

# First hidden layer
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

# Second hidden layer
model.add(Dense(64, activation='relu'))

# Output layer (for binary classification: win or not win)
model.add(Dense(1, activation='sigmoid'))


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


In [ ]:
# Save the model
model.save('tic-tac-toe.model')


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('tic-tac-toe.model')


In [ ]:
import numpy as np

# Select a few random rows from the test dataset
random_indices = np.random.choice(X_test.shape[0], size=5, replace=False)
X_random = X_test[random_indices]
y_random = y_test[random_indices]

# Make predictions
predictions = model.predict(X_random)

# Convert predictions to binary (0 or 1)
predictions = (predictions > 0.5).astype(int)

# Display predictions and actual values
for i in range(len(predictions)):
    print(f"Prediction: {'X wins' if predictions[i] == 1 else 'X loses'} - Actual: {'X wins' if y_random[i] == 1 else 'X loses'}")


In [ ]:
# your code here

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a deeper Sequential model with more layers
model = Sequential()

# Input layer and first hidden layer
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# Additional hidden layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model with custom learning rate
from tensorflow.keras.optimizers import Adam

# Create a custom Adam optimizer with a lower learning rate
optimizer = Adam(learning_rate=0.001)  # You can try other values, like 0.0005 or 0.002

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model for more epochs
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model after training
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
second is increasing the number of epochs allows the model more opportunities to learn from the data. 